# 2. Linear Regression-线性回归Pytorch实现
## 2.1 导入所需包和模块

In [10]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt 
import numpy as np
import random
import torch.nn as nn

## 2.2 生成人工数据集 
与LinearRegression.ipynb中数据集产生一样

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

## 2.3 读取数据集

In [3]:
import torch.utils.data as Data

batch_size = 10

dataset = Data.TensorDataset(features, labels)
# 每次训练读取小批量数据集
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

打印第一个小批量数据集:

In [8]:
for X,y in data_iter:
    print(X, y)
    break

tensor([[ 0.8606, -0.0050],
        [ 0.2138, -0.3199],
        [-3.4722,  1.9843],
        [-0.8635, -0.6451],
        [ 0.6306,  0.5322],
        [ 1.0577,  1.8457],
        [-0.8219,  0.9678],
        [-0.0929, -0.8129],
        [ 0.0227, -1.2095],
        [ 1.3182,  0.0140]])tensor([ 5.9447,  5.7175, -9.4954,  4.6728,  3.6597,  0.0426, -0.7450,  6.7774,
         8.3565,  6.7945])


## 2.4 定义模型

In [26]:
class LinearNet(nn.Module):
    def __init__(self, n_features):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_features, 1)
    
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


查看模型可学习参数:

In [27]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.4158,  0.5881]], requires_grad=True)
Parameter containing:
tensor([0.0613], requires_grad=True)


## 2.5 初始化模型参数

In [28]:
from torch.nn import init

init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0)

Parameter containing:
tensor([0.], requires_grad=True)

## 2.6 定义损失函数

In [29]:
loss = nn.MSELoss()

## 2.7 定义优化算法

In [30]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


2.8 训练模型

In [31]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 更新参数之前，梯度清零
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000294
epoch 2, loss: 0.000127
epoch 3, loss: 0.000087


打印出训练介绍后参数和真实参数:

In [34]:
dense = net.linear
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4]Parameter containing:
tensor([[ 2.0012, -3.4002]], requires_grad=True)
4.2Parameter containing:
tensor([4.2002], requires_grad=True)
